In [ ]:
# Extraction des résumés en français et en anglais, et le lien vers le mémoire pdf, et le lien permanent.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# URL de la page à scraper
url = "https://www.ressources-actuarielles.net/memoires"

# Fonction pour extraire les données
def extraire_memoires():
    # Récupération de la page
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Erreur lors de la récupération de la page: {response.status_code}")
        return None
    
    # Parsing du HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Recherche des articles de mémoires
    articles = soup.find_all('article', class_='node--type-memoire')
    
    resultats = []
    
    for article in articles:
        # Extraction du titre
        titre_element = article.find('h2', class_='node__title')
        titre = titre_element.text.strip() if titre_element else "Titre non disponible"
        
        # Extraction du lien permanent
        lien_permanent = None
        if titre_element and titre_element.find('a'):
            lien_permanent = "https://www.ressources-actuarielles.net" + titre_element.find('a')['href']
        
        # Extraction du lien PDF
        lien_pdf = None
        pdf_element = article.find('a', href=re.compile(r'\.pdf$'))
        if pdf_element:
            lien_pdf = "https://www.ressources-actuarielles.net" + pdf_element['href']
        
        # Extraction des résumés
        resume_fr = ""
        resume_en = ""
        
        # Chercher les sections de résumé
        resume_sections = article.find_all('div', class_='field--type-text-with-summary')
        
        for section in resume_sections:
            # Vérifier si c'est un résumé français ou anglais
            label = section.find('div', class_='field__label')
            if label:
                label_text = label.text.strip().lower()
                content = section.find('div', class_='field__item')
                
                if content:
                    if 'français' in label_text or 'resume' in label_text:
                        resume_fr = content.text.strip()
                    elif 'anglais' in label_text or 'abstract' in label_text:
                        resume_en = content.text.strip()
        
        # Ajouter les données extraites à la liste des résultats
        resultats.append({
            'Titre': titre,
            'Lien Permanent': lien_permanent,
            'Lien PDF': lien_pdf,
            'Résumé Français': resume_fr,
            'Résumé Anglais': resume_en
        })
        
        # Pause pour éviter de surcharger le serveur
        time.sleep(0.5)
    
    return resultats

# Exécution de la fonction d'extraction
memoires = extraire_memoires()

# Création d'un DataFrame pandas pour une meilleure visualisation
if memoires:
    df = pd.DataFrame(memoires)
    print(f"Nombre de mémoires extraits: {len(df)}")
    
    # Affichage des 5 premiers résultats
    print("\nAperçu des données extraites:")
    print(df[['Titre', 'Lien Permanent', 'Lien PDF']].head())
    
    # Sauvegarde des résultats dans un fichier CSV
    df.to_csv('memoires_actuariels.csv', index=False, encoding='utf-8-sig')
    print("\nLes données ont été sauvegardées dans 'memoires_actuariels.csv'")
else:
    print("Échec de l'extraction des données.")